# PM Tiles demo

Denne notebooken er ment å vise frem kapabilitetene i PM Tiles formatet.

## Systemkrav

Demoen krever noen verktøy som ikke kan håndteres isolert i et python miljø,
for at dette skal være så portabelt som mulig er det lagt opp til å bruke Docker
til å håndtere miljøet for disse verktøyene.

Blokken under sjekker at Docker og Docker-compose er installert på vertssystemet.

In [1]:
import subprocess
from typing import List

def execute_command(cmd: List[str], **kwargs):
    return subprocess.run(
        cmd,
        capture_output=True,
        text=True,
        **kwargs
    )

system_requirements = [
    ["docker-compose", "--version"],
    ["docker", "--version"],
]

for requirement in system_requirements:
    assert execute_command(requirement, check=False).returncode == 0, f"System requirement {requirement} is not satisfied"

## Input data



### Raster


![nedlasting](../../public/img/geonorge-n50-raster-krs.png)

Last ned filen som vist over, pakk ut og flytt .tif filen til `raster/data/N50_raster_2024.tif`


### Vektor 

![nedlasting](../../public/img/geonorge-n50-vektor-agder.png)

Last ned filen som vist over, pakk ut og flytt .gdb filen til `vector/data/N50_vektor_agder.gdb`

Først sjekker vi at input filene finnes som forventet.

In [22]:
import utils

assert utils.path_exists("raster/data/N50_raster_2024.tif"), f"Mangler inputfil raster/data/N50_raster_2024.tif"
assert utils.path_exists("vector/data/N50_vektor_agder.gdb"), f"Mangler inputfil vector/data/N50_vektor_agder.gdb"

## Konverterer raster data

Merk at her brukes python komandolinjeverktøyet `rio-pmtiles` som installeres via requirements.txt filen.

```bash
# Etter at virtuelt python miljø er aktivert.
pip install -r requirements.txt
```

In [3]:
# Oppretter mappe for output
utils.create_dir("raster/out")


Directory 'raster/out' created successfully.


In [4]:
# Genererer pmtiles
!rio pmtiles raster/data/N50_raster_2024.tif raster/out/N50_raster_2024.pmtiles --format PNG

100%|██████████████████████████████████████| 1425/1425 [00:12<00:00, 116.45it/s]


## Konverterer vektor data

In [5]:
# Oppretter mappe for output
utils.create_dir("vector/out")

Directory 'vector/out' created successfully.


### Dockerfil

For at demoen skal være relativt portabel skjer selve transformasjonen fra gdb til pmtiles
i en docker container gjennom en rekke transformasjoner ved hjelp av flere verktøy.

Format transformasjon:

gdb => geojson => mbtiles => pmtiles

Verktøy:

- ogr2ogr: Konverterer gdb til geojson (gdal)
- tippecanoe: Konverterer geojson til mbtiles
- pmtiles: Konverterer mbtiles til pmtiles


Det er mulig å kombinere flere lag i samme .pmtiles fil, men ved bruk av `vector/build.sh` scriptet fungerte
ikke dette helt som forventet. Lagene blir opprettet, men ved lavere zoom ser ikke kartet pent ut. Videreutvikling
oppfordres.

In [6]:
# Bygger docker image med gdal, tippecanoe og pmtiles
!docker build -t pmtiles -f vector/Dockerfile .

[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0.2s (1/2)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 953B                                       0.0s
 => [internal] load metadata for docker.io/library/ubuntu:latest           0.2s
[+] Building 0.3s (1/2)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 953B                                       0.0s
 => [internal] load metadata for docker.io/library/ubuntu:latest           0.3s
[+] Building 0.5s (1/2)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 953B                                       0.0s
 => [internal] load metadata for docker.

Starter en container for å kunne kopiere lokale filer over til containeren og kjøre konverteringsscriptet
i det isolerte miljøet.

In [7]:
!docker run --name pmtiles-converter -d pmtiles

dae947669be3d333436457ca7e30e13ff0a92aa2e737c6c0138564a9940b2b02


Kopierer kildedata inn i containeren.

In [8]:
!docker cp ./vector/data/N50_vektor_agder.gdb pmtiles-converter:/app/N50_vektor_agder.gdb 

Successfully copied 397MB to pmtiles-converter:/app/N50_vektor_agder.gdb


Konverterer samferdsel senterlinje laget til pmtiles

In [9]:
!docker exec pmtiles-converter bash gdb_to_pmtiles.sh N50_vektor_agder.gdb N50_Samferdsel_senterlinje N50_Samferdsel_senterlinje.pmtiles

🧩 Converting GDB to GeoJSON...
🗺️ Creating MBTiles with tippecanoe...
81278 features, 7790042 bytes of geometry, 5943650 bytes of separate metadata, 1645464 bytes of string pool
Choosing a maxzoom of -z8 for features about 1074 feet (328 meters) apart
Choosing a maxzoom of -z11 for resolution of about 193 feet (59 meters) within features
tile 3/4/2 size is 539081 with detail 12, >500000    
Going to try keeping the sparsest 83.48% of the features to make it fit
tile 3/4/2 size is 522622 with detail 12, >500000    
Going to try keeping the sparsest 71.88% of the features to make it fit
tile 3/4/2 size is 506124 with detail 12, >500000    
Going to try keeping the sparsest 63.91% of the features to make it fit
tile 4/8/4 size is 816407 with detail 12, >500000    
Going to try keeping the sparsest 55.12% of the features to make it fit
tile 4/8/4 size is 644298 with detail 12, >500000    
Going to try keeping the sparsest 38.50% of the features to make it fit
tile 4/8/4 size is 506758 with

Kopierer pmtiles filen ut av containeren

In [10]:
!docker cp pmtiles-converter:/app/N50_Samferdsel_senterlinje.pmtiles ./vector/out/N50_Samferdsel_senterlinje.pmtiles

Successfully copied 12.2MB to /home/kluesp/projects/skygeo/src/pmtiles/vector/out/N50_Samferdsel_senterlinje.pmtiles


Konverterer bygninger og anlegg område laget til pmtiles

In [11]:
!docker exec pmtiles-converter bash gdb_to_pmtiles.sh N50_vektor_agder.gdb N50_BygningerOgAnlegg_omrade N50_BygningerOgAnlegg_omrade.pmtiles

🧩 Converting GDB to GeoJSON...
🗺️ Creating MBTiles with tippecanoe...
2681 features, 190351 bytes of geometry, 103754 bytes of separate metadata, 113091 bytes of string pool
Choosing a maxzoom of -z8 for features about 1799 feet (549 meters) apart
Choosing a maxzoom of -z11 for resolution of about 146 feet (44 meters) within features
  99.9%  11/1067/613  
📦 Converting MBTiles to PMTiles...
2025/08/18 07:56:32 convert.go:159: Pass 1: Assembling TileID set
2025/08/18 07:56:32 convert.go:190: Pass 2: writing tiles
 100% |█████████████████████████████████████████| (200/200, 49337 it/s)        
2025/08/18 07:56:32 convert.go:244: # of addressed tiles:  200
2025/08/18 07:56:32 convert.go:245: # of tile entries (after RLE):  200
2025/08/18 07:56:32 convert.go:246: # of tile contents:  200
2025/08/18 07:56:32 convert.go:269: Total dir bytes:  544
2025/08/18 07:56:32 convert.go:270: Average bytes per addressed tile: 2.72
2025/08/18 07:56:32 convert.go:239: Finished in  5.606638ms
✅ Done! PMTil

Kopierer pmtiles filen ut av containeren

In [12]:
!docker cp pmtiles-converter:/app/N50_BygningerOgAnlegg_omrade.pmtiles ./vector/out/N50_BygningerOgAnlegg_omrade.pmtiles

Successfully copied 528kB to /home/kluesp/projects/skygeo/src/pmtiles/vector/out/N50_BygningerOgAnlegg_omrade.pmtiles


Stopper og fjerner containeren etter at konverteringen er ferdig.

In [13]:
!docker stop pmtiles-converter
!docker rm pmtiles-converter

pmtiles-converter
pmtiles-converter


Fjerner docker image

In [14]:
!docker rmi pmtiles

Untagged: pmtiles:latest
Deleted: sha256:76975773dfd1d45e64823fa55ee2f01fca6a8ac38f2ba5a16aeed928fd10d8bd


# Demo

I denne delen bruker vi `docker-compose` til å kjøre en lokal demo hvor pmtiles filene hostes
over nettet og en veldig enkel web-app leser data fra filene.

Demoen består av to applikasjoner:

- Web appen som kjører i nettleseren.
- En ngix container som hoster de tre pmtiles filene vi har generert.

Mens docker compose blokken under kjører kan web appen åpnes på 
[http://localhost:3000](http://localhost:3000).
I appen vil man se de tre filene vi har generert som lag, det kreves ingen back-end prosessering utover
å tilgjengeliggjøre filene over internett. Kildekoden for web appen finnes under `client/` mappen.
Ved å inspisere nettverkstrafikken i nettleseren kan man observere 
hvordan range headeren brukes til å spørre ut deler av filene.

(Demo blokken må stoppes manuelt)

In [28]:
!docker compose up --build

Compose can now delegate builds to bake for better performance.
 To do so, set COMPOSE_BAKE=true.
[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0.2s (11/13)                                        docker:default
 => [backend internal] load build definition from Dockerfile               0.0s
 => => transferring dockerfile: 1.04kB                                     0.0s
 => [frontend internal] load build definition from Dockerfile              0.0s
 => => transferring dockerfile: 454B                                       0.0s
 => [backend internal] load metadata for docker.io/library/nginx:latest    0.0s
 => [frontend internal] load metadata for docker.io/library/nginx:stable-  0.2s
 => [frontend internal] load metadata for docker.io/library/node:18        0.2s
 => [backend internal] load .dockerignore                                  0.0s
 => => transferring context: 2B                                            0.0s
 => [backend 1/4] FROM

Fjerner demo containere og images

In [29]:
!docker compose rm -f -s -v

[+] Stopping 2/2
 ✔ Container pmtiles-frontend-1  Stopped                                   0.0s 
 ✔ Container pmtiles-backend-1   Stopped                                   0.0s 
Going to remove pmtiles-backend-1, pmtiles-frontend-1
[+] Removing 2/2
 ✔ Container pmtiles-backend-1   Removed                                   0.0s 
 ✔ Container pmtiles-frontend-1  Removed                                   0.0s 
